In [1]:
import numpy as np
np.random.seed(8)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

import warnings
warnings.filterwarnings('ignore')

import os
os.environ['OMP_NUM_THREADS'] = '4'

train = pd.read_csv('train.csv')
test_cm = pd.read_csv('test.csv')
test_lb = pd.read_csv('test_labels.csv')
test_all = pd.merge(test_cm, test_lb, on='id')
test = test_all[test_all['toxic'] != -1]

Using TensorFlow backend.


In [5]:
X_train = train["comment_text"].values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].values
y_test = test[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

In [6]:
vocab = 100000
maxlen = 200
embed_size = 300

In [7]:
t = text.Tokenizer(num_words=vocab)
t.fit_on_texts(list(X_train) + list(X_test))

In [8]:
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [9]:
EMBEDDING_FILE = 'crawl-300d-2M.vec'
embeddings_index = dict()
f = open(EMBEDDING_FILE)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s pre-trained words' % len(embeddings_index))


Loaded 2000000 pre-trained words


In [10]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab, embed_size))
for word, i in t.word_index.items():
    if i >= vocab: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [11]:
num_filters = 32

inp = Input(shape=(maxlen, ))
x = Embedding(vocab, embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False)(inp)
x = SpatialDropout1D(0.5)(x)
x = Reshape((maxlen, embed_size, 1))(x)

conv_0 = Conv2D(num_filters, kernel_size=(2, embed_size), activation='relu')(x)
maxpool_0 = MaxPool2D(pool_size=(maxlen - 1, 1))(conv_0)
conv_1 = Conv2D(num_filters, kernel_size=(3, embed_size), activation='relu')(x)
maxpool_1 = MaxPool2D(pool_size=(maxlen - 2, 1))(conv_1)
conv_2 = Conv2D(num_filters, kernel_size=(4, embed_size), activation='relu')(x)
maxpool_2 = MaxPool2D(pool_size=(maxlen - 3, 1))(conv_2)
conv_3 = Conv2D(num_filters, kernel_size=(5, embed_size), activation='relu')(x)
maxpool_3 = MaxPool2D(pool_size=(maxlen - 4, 1))(conv_3)
#conv_4 = Conv2D(num_filters, kernel_size=(6, embed_size), activation='relu')(x)
#maxpool_4 = MaxPool2D(pool_size=(maxlen - 5, 1))(conv_4)

y = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, maxpool_3])
y = Flatten()(y)
y = Dropout(0.1)(y)

outp = Dense(6, activation="sigmoid")(y)

model = Model(inputs=inp, outputs=outp)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 200)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 200, 300)      30000000                                     
____________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDrop (None, 200, 300)      0                                            
____________________________________________________________________________________________________
reshape_1 (Reshape)              (None, 200, 300, 1)   0                                            
___________________________________________________________________________________________

In [12]:
x_trainS, x_val, y_trainS, y_val = train_test_split(x_train, y_train, train_size=0.80, random_state=8)

In [13]:
class RocAuc(Callback):

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.validation_data[0], verbose=0)
        score = roc_auc_score(self.validation_data[1], y_pred)
        print("\n roc_auc - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [14]:
from keras.callbacks import ModelCheckpoint
batch_size=256
epochs = 3
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.pre_trained.conv2.testscore2.hdf5', verbose=1, save_best_only=True)
RocAuc = RocAuc()

In [15]:
h = model.fit(x_trainS, y_trainS, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val),
                 callbacks=[checkpointer, RocAuc], verbose=2)

Train on 127656 samples, validate on 31915 samples
Epoch 1/3
Epoch 00000: val_loss improved from inf to 0.05078, saving model to saved_models/weights.best.pre_trained.conv2.testscore2.hdf5

 roc_auc - epoch: 1 - score: 0.967924 

684s - loss: 0.0835 - acc: 0.9748 - val_loss: 0.0508 - val_acc: 0.9817
Epoch 2/3
Epoch 00001: val_loss improved from 0.05078 to 0.04760, saving model to saved_models/weights.best.pre_trained.conv2.testscore2.hdf5

 roc_auc - epoch: 2 - score: 0.983512 

681s - loss: 0.0558 - acc: 0.9801 - val_loss: 0.0476 - val_acc: 0.9823
Epoch 3/3
Epoch 00002: val_loss improved from 0.04760 to 0.04593, saving model to saved_models/weights.best.pre_trained.conv2.testscore2.hdf5

 roc_auc - epoch: 3 - score: 0.986378 

816s - loss: 0.0515 - acc: 0.9812 - val_loss: 0.0459 - val_acc: 0.9828


In [18]:
y_pred = model.predict(x_test, batch_size=2048)
score = roc_auc_score(y_test, y_pred)
print("\n roc_auc score: %.6f \n" % (score))


 roc_auc score: 0.977971 

